# Test Correspondences

In [12]:
%matplotlib widget
import numpy as np
import os
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import planeslam.io as io
from planeslam.general import NED_to_ENU
from planeslam.scan import velo_pc_to_scan
from planeslam.registration import extract_corresponding_features, get_correspondences
from planeslam.point_cloud import velo_preprocess

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Read in airsim LiDAR and pose data

In [13]:
# Read in point cloud data
pcpath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_12_2022', 'flightroom', 'run_1', 'pcs')
PCs = []
for i in range(len(os.listdir(pcpath))):  # ignore first 200 and last 200 frames
    filename = pcpath+'\pc_'+str(i)+'.npy'
    PC = np.load(filename)
    PCs.append(PC)

In [14]:
# Read in pose data
posepath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_12_2022', 'flightroom', 'run_1', 'poses')
poses = []
for i in range(len(os.listdir(pcpath))):  # ignore first 200 and last 200 frames
    filename = posepath+'\pose_'+str(i)+'.npy'
    pose = np.load(filename)
    poses.append(pose)

Extract planes 

In [23]:
# Plot 2 scans
source_idx = 100
target_idx = 99
source = velo_pc_to_scan(velo_preprocess(PCs[source_idx], poses[source_idx]))
target = velo_pc_to_scan(velo_preprocess(PCs[target_idx], poses[target_idx]))

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in source.plot_trace(show_normals=False):
    fig.add_trace(t, row=1, col=1)

for t in target.plot_trace(show_normals=False):
    fig.add_trace(t, row=1, col=2)

fig.update_layout(width=1600, height=500, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

correspondences = get_correspondences(source, target)
print(correspondences)

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 9), (9, 10), (10, 2), (11, 13), (12, 11), (13, 0), (14, 10), (15, 15), (16, 16)]


In [24]:
source.remove_small_planes(area_thresh=0.1)
fig = go.Figure(data=source.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
n = len(source.planes) # source P
m = len(target.planes) # target Q
score_mat = np.zeros((n,m))
for i in range(n):
    for j in range(m):
        n1 = source.planes[i].normal
        n2 = target.planes[j].normal
        c1 = source.planes[i].center
        c2 = target.planes[j].center
        a1 = source.planes[i].area()
        a2 = target.planes[j].area()
        score_mat[i,j] = 100 * np.linalg.norm(n1 - n2) + np.linalg.norm(c1 - c2) + 0.1 * np.abs(a1 - a2)
        #score_mat[i,j] = np.linalg.norm(c1 - c2)

In [ ]:
from scipy.optimize import linear_sum_assignment

matches = linear_sum_assignment(score_mat)
matches

In [18]:
from planeslam.registration import robust_GN_register

R_hat, t_hat = robust_GN_register(source, target)

print(R_hat)
print(t_hat)

final translation loss:  0.014967072845914302
translation residuals:  [[ 0.00334722]
 [ 0.03465687]
 [-0.0389035 ]
 [-0.00891457]
 [ 0.01041472]
 [-0.02723419]
 [-0.04587644]
 [ 0.03111979]
 [ 0.00269682]
 [ 0.017353  ]
 [ 0.02786027]
 [ 0.0151155 ]
 [-0.02911651]
 [-0.06758218]
 [ 0.02684558]
 [ 0.01784208]
 [-0.00604403]
 [ 0.02085217]]
[[ 0.99981773  0.01868703  0.00391088]
 [-0.01867354  0.99981966 -0.00345726]
 [-0.00397478  0.0033836   0.99998638]]
[[0.00714994]
 [0.02917369]
 [0.00426312]]


In [ ]:
target.basis @ source.basis.T

In [19]:
from planeslam.geometry.util import quat_to_R

R_1 = quat_to_R(poses[target_idx][3:])
R_2 = quat_to_R(poses[source_idx][3:])
R_gt = R_2 @ R_1.T

t_gt = poses[source_idx][:3] - poses[target_idx][:3]
print(R_gt)
print(t_gt)

[[ 9.99999999e-01 -1.96435465e-05 -2.73755649e-05]
 [ 1.96421101e-05  9.99999998e-01 -5.24688161e-05]
 [ 2.73765955e-05  5.24682783e-05  9.99999998e-01]]
[-1.33514404e-05  3.42354178e-06 -4.41074371e-06]


In [ ]:
source.transform(R_hat, t_hat.flatten())

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in source.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=1)

for t in target.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=2)

fig.update_layout(width=1200, height=500, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

In [ ]:
fig = go.Figure(data=source.plot_trace()+target.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()